<a href="https://colab.research.google.com/github/ITDeema/DataScience/blob/main/DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update -y
!apt-get install -y chromium-chromedriver
!pip install -q selenium pandas openpyxl


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,115 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.2 kB]

In [3]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=15d44fac7dcfcae5661d213db6fb15a740a064002cdc366cecde8f1297765b17
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [6]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    # Makkah
    "Hotels in Makkah",
    "Hotels near Masjid al Haram",
    "Hotels in Ajyad Makkah",
    "Hotels in Aziziyah Makkah",
    "Hotels in Al Misfalah Makkah",
    "Hotels in Al Shubaikah Makkah",
    "Hotels in Al Naseem Makkah",
    "Hotels in Kudai Makkah",
    "Hotels in Jarwal Makkah",

    # Madinah
    "Hotels in Madinah",
    "Hotels near Al Masjid an Nabawi",
    "Hotels in Central Madinah",
    "Hotels in Al Haram Madinah",
    "Hotels in Quba Madinah",
    "Hotels in Al Azhari Madinah",
    "Hotels in Al Aridh Madinah",
    "Hotels in Bani Khidrah Madinah"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40, 60, 80]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "High spiritual tourism season during Ramadan"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Peak pilgrimage demand period"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj booking increase period"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular travel season with moderate hotel demand"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None


                city = "Madinah" if "Madinah" in q else "Makkah"

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "City": city,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_final = pd.DataFrame(all_hotels)

df_final = df_final.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total rows collected:", len(df_final))

df_final.to_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv", index=False)

df_final.head()


Total rows collected: 1137


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,نجوم السعد 3,‏٨٦٤ ر.س.‏,NaN,NaN,"Lat: 21.372940063476562, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,نجوم السعد 6,‏٩٠٢ ر.س.‏,2.2,3.0,"Lat: 21.372928619384766, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,ضيافة الطلائع للشقق المخدومة - One-Bedroom Apa...,‏٣٬٣٠٠ ر.س.‏,NaN,NaN,"Lat: 21.422340393066406, Lng: 39.848140716552734",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق الميقات اجياد,None,3.7,387.0,"Lat: 21.4197991, Lng: 39.828957599999995",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan
4,Hotel mekkah,‏٧٣١ ر.س.‏,5.0,3.0,"Lat: 21.418908599999998, Lng: 39.8290042",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan


In [7]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    "Hotels in Taif",
    "Hotels in Taif for Hajj visitors",
    "Hotels in Taif near Miqat",
    "Umrah visitor hotels in Taif",
    "Budget hotels in Taif for pilgrims",
    "Luxury hotels in Taif during Hajj season",
    "Hotels in Taif near Al Sail Al Kabir Miqat"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "Ramadan spiritual tourism season affecting Taif hotels"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Hajj season overflow demand with pilgrims staying in Taif"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj increasing accommodation demand in Taif"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular Umrah related travel near Makkah region"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "City": "Taif",
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_taif = pd.DataFrame(all_hotels)

df_taif = df_taif.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total Taif rows:", len(df_taif))

df_taif.to_csv("taif_hajj_umrah_2026.csv", index=False)

df_taif.head()

Total Taif rows: 323


,Hotel Name,City,Price (SAR),Rating,Reviews,Address,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,فندق وشاليهات الخزامى السياحي,Taif,‏١٩٢ ر.س.‏,3.8,1099.0,"Lat: 21.300456, Lng: 40.4335465",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
1,فندق كيان الطائف,Taif,‏٢٧١ ر.س.‏,4.3,1046.0,"Lat: 21.2944004, Lng: 40.4350932",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
2,سديم الفجر للشقق الفندقية - Sadeem Alfajr Hote...,Taif,‏٢٠٠ ر.س.‏,3.6,843.0,"Lat: 21.2882806, Lng: 40.416078500000005",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
3,سفن جاردنز-البيعة,Taif,‏٣٥٠ ر.س.‏,4.4,1584.0,"Lat: 21.284342499999998, Lng: 40.4451623",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
4,توت هاوس الوسام,Taif,‏٢٢٣ ر.س.‏,4.4,1126.0,"Lat: 21.233142600000004, Lng: 40.3731541",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...


In [8]:
import pandas as pd

df_main = pd.read_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv")
df_taif = pd.read_csv("taif_hajj_umrah_2026.csv")

print("Rows before merge:")
print("Main file:", len(df_main))
print("Taif file:", len(df_taif))

df_main.columns = df_main.columns.str.strip()
df_taif.columns = df_taif.columns.str.strip()

df_combined = pd.concat([df_main, df_taif], ignore_index=True, sort=False)

df_combined = df_combined.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Final merged rows:", len(df_combined))

df_combined.to_csv("FINAL_HAJJ_UMRAH_ALL_CITIES_2026.csv", index=False)

df_combined.head()


Rows before merge:
Main file: 1137
Taif file: 323
Final merged rows: 1460


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,نجوم السعد 3,‏٨٦٤ ر.س.‏,NaN,NaN,"Lat: 21.372940063476562, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,نجوم السعد 6,‏٩٠٢ ر.س.‏,2.2,3.0,"Lat: 21.372928619384766, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,ضيافة الطلائع للشقق المخدومة - One-Bedroom Apa...,‏٣٬٣٠٠ ر.س.‏,NaN,NaN,"Lat: 21.422340393066406, Lng: 39.848140716552734",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق الميقات اجياد,NaN,3.7,387.0,"Lat: 21.4197991, Lng: 39.828957599999995",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan
4,Hotel mekkah,‏٧٣١ ر.س.‏,5.0,3.0,"Lat: 21.418908599999998, Lng: 39.8290042",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan


In [70]:
import numpy as np

df_raw = pd.read_csv("FINAL_HAJJ_UMRAH_ALL_CITIES_2026.csv")

# إنشاء نسخة للعمل عليها
df = df_raw.copy()

In [71]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Hotel Name         1460 non-null   object 
 1   Price (SAR)        810 non-null    object 
 2   Rating             1376 non-null   float64
 3   Reviews            1376 non-null   float64
 4   Address            1460 non-null   object 
 5   City               1460 non-null   object 
 6   Check-in           1460 non-null   object 
 7   Check-out          1460 non-null   object 
 8   Search Area        1460 non-null   object 
 9   Season_Type        1460 non-null   object 
 10  Religious_Context  1460 non-null   object 
dtypes: float64(2), object(9)
memory usage: 125.6+ KB


,Rating,Reviews
count,1376.000000,1376.000000
mean,4.108913,3665.493459
std,0.465629,10098.426717
min,1.800000,1.000000
25%,3.800000,145.000000
50%,4.100000,573.000000
75%,4.500000,1947.000000
max,5.000000,117062.000000


In [72]:
missing_table = pd.DataFrame({
    "Missing Count": df.isnull().sum(),
    "Missing %": df.isnull().mean() * 100
})

missing_table.sort_values("Missing %", ascending=False)

,Missing Count,Missing %
Price (SAR),650,44.520548
Rating,84,5.753425
Reviews,84,5.753425
Hotel Name,0,0.000000
Address,0,0.000000
City,0,0.000000
Check-in,0,0.000000
Check-out,0,0.000000
Search Area,0,0.000000
Season_Type,0,0.000000


In [73]:
# فحص عدد التكرارات بناءً على الموقع وتاريخ الدخول بدلاً من الاسم
duplicate_count = df.duplicated(subset=['Address', 'Check-in', 'Check-out']).sum()
print(f"Number of duplicates found: {duplicate_count}")

# حذف التكرارات
df.drop_duplicates(subset=['Address', 'Check-in', 'Check-out'], inplace=True)
df.drop_duplicates(inplace=True)

Number of duplicates found: 3


In [74]:
import re

def arabic_to_english_number(text):
    arabic_numbers = "٠١٢٣٤٥٦٧٨٩"
    english_numbers = "0123456789"
    translator = str.maketrans(arabic_numbers, english_numbers)
    return text.translate(translator)

def clean_price(value):
    if pd.isna(value):
        return None
    value = str(value)
    value = arabic_to_english_number(value)
    value = re.sub(r"[^\d]", "", value)   # ← هذا هو التغيير
    if value == "":
        return None
    return float(value)

df["Price (SAR)"] = df["Price (SAR)"].apply(clean_price)

print(df["Price (SAR)"])

0        864.0
1        902.0
2       3300.0
3          NaN
4        731.0
         ...  
1455       NaN
1456       NaN
1457       NaN
1458       NaN
1459    1136.0
Name: Price (SAR), Length: 1457, dtype: float64


In [80]:
df["Price (SAR)"] = df["Price (SAR)"].fillna(df["Price (SAR)"].median())
df["Rating"] = df["Rating"].fillna(df["Rating"].mean()).round(1)
df["Reviews"] = df["Reviews"].fillna(0)

print(df.isnull().sum())

Hotel Name           0
Price (SAR)          0
Rating               0
Reviews              0
Address              0
City                 0
Check-in             0
Check-out            0
Search Area          0
Season_Type          0
Religious_Context    0
Latitude             0
Longitude            0
dtype: int64


In [81]:
# 1. استخراج الإحداثيات مباشرة في الأعمدة الجديدة
# أضفنا علامة -? تحسباً لو كان هناك إحداثيات بالسالب (أفضل ممارسة في علم البيانات)
df[["Latitude", "Longitude"]] = df["Address"].str.extract(r"Lat:\s*(-?[\d\.]+),\s*Lng:\s*(-?[\d\.]+)")

# 2. تحويل الأعمدة إلى أرقام عشرية
df["Latitude"] = pd.to_numeric(df["Latitude"])
df["Longitude"] = pd.to_numeric(df["Longitude"])

# 3. التأكد من النتيجة
print(df[["Address", "Latitude", "Longitude"]].head())

                                            Address   Latitude  Longitude
0  Lat: 21.372940063476562, Lng: 39.841705322265625  21.372940  39.841705
1  Lat: 21.372928619384766, Lng: 39.841705322265625  21.372929  39.841705
2  Lat: 21.422340393066406, Lng: 39.848140716552734  21.422340  39.848141
3          Lat: 21.4197991, Lng: 39.828957599999995  21.419799  39.828958
4          Lat: 21.418908599999998, Lng: 39.8290042  21.418909  39.829004


In [82]:
df["Check-in"] = pd.to_datetime(df["Check-in"])
df["Check-out"] = pd.to_datetime(df["Check-out"])
print(df[["Check-in", "Check-out"]])

       Check-in  Check-out
0    2026-03-10 2026-03-15
1    2026-03-10 2026-03-15
2    2026-03-10 2026-03-15
3    2026-03-10 2026-03-15
4    2026-03-10 2026-03-15
...         ...        ...
1455 2026-07-01 2026-07-06
1456 2026-07-01 2026-07-06
1457 2026-07-01 2026-07-06
1458 2026-07-01 2026-07-06
1459 2026-07-01 2026-07-06

[1457 rows x 2 columns]


In [83]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 1457 entries, 0 to 1459
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Hotel Name         1457 non-null   object        
 1   Price (SAR)        1457 non-null   float64       
 2   Rating             1457 non-null   float64       
 3   Reviews            1457 non-null   float64       
 4   Address            1457 non-null   object        
 5   City               1457 non-null   object        
 6   Check-in           1457 non-null   datetime64[ns]
 7   Check-out          1457 non-null   datetime64[ns]
 8   Search Area        1457 non-null   object        
 9   Season_Type        1457 non-null   object        
 10  Religious_Context  1457 non-null   object        
 11  Latitude           1457 non-null   float64       
 12  Longitude          1457 non-null   float64       
dtypes: datetime64[ns](2), float64(5), object(6)
memory usage: 159.4+ KB


,0
Hotel Name,0
Price (SAR),0
Rating,0
Reviews,0
Address,0
City,0
Check-in,0
Check-out,0
Search Area,0
Season_Type,0


In [84]:
df.to_csv("CLEANED_HAJJ_UMRAH.csv", index=False)